First, we will import the needed packages such as Tensorflow, NumPy, Pandas, Seaborn, etc. We will also create a path variable, and the Pandas dataframes for both training and testing. We will create a BASE_PATH to help with keeping track of the data file structure. This notebook takes advantage of the GTSRB dataset posted to Kaggle. For more information about the original dataset, please see the [INI GTSRB webpage.](https://benchmark.ini.rub.de/gtsrb_dataset.html)

In [ ]:
# Packge Imports
import tensorflow as tf
import numpy as np 
import pandas as pd
import seaborn as sns
import cv2
import matplotlib.pyplot as plt
from sklearn import metrics, utils
from PIL import Image
import os


%matplotlib inline


# create a base path
BASE_PATH = "../input/gtsrb-german-traffic-sign/"        

# read the Training.csv and Test.csv files into dataframes
training_dataframe = pd.read_csv(BASE_PATH + 'Train.csv')
test_dataframe = pd.read_csv(BASE_PATH + 'Test.csv')

training_dataframe.head() # show the head of the training dataframe


Next, we will get the number of classes using the Pandas nunique function and store that into the num_classes variable. This will be used later when we build the neural network and are defining the softmax layer. We will also randomly sample a few images to get a feel for what the data looks like.

In [ ]:
NUM_CLASSES = training_dataframe['ClassId'].nunique() # Calculate the number of unique classes
TRAIN_LEN = training_dataframe['ClassId'].count()
print("Number of Classes: ", NUM_CLASSES) # debug print, dataset contains 43 classes


# lets look at some random images
sample = plt.subplots(5,5, figsize=(15,10))
for i in range(25):
    plt.subplot(5,5,i+1) # Plot A Random Image
    plt.imshow(plt.imread(BASE_PATH+training_dataframe["Path"][np.random.randint(TRAIN_LEN)]))

plt.show()

There are 43 classes of images. After searching online for some time, I was able to find a couple of notebooks that had the class names, which will be added in the next section. In addition to adding the class names, we find that the randomly sampled images are of a different shape/size. This means that we will want to find an appropraite size for our flow_from_dataframe generator, which will be defined in a later section.

In this next block, in addition to creating a class names dictionary, we will reduce the training_dataframe to the image path and the class id, and will convert the class id to a string type. Based on the organization of the dataset, we will need to shuffle the dataframe to ensure training is more effective.

In [ ]:
CLASS_NAMES = { 0:'Speed limit (20km/h)', 1:'Speed limit (30km/h)', 2:'Speed limit (50km/h)', 
            3:'Speed limit (60km/h)', 4:'Speed limit (70km/h)', 5:'Speed limit (80km/h)', 
            6:'End of speed limit (80km/h)', 7:'Speed limit (100km/h)', 8:'Speed limit (120km/h)', 9:'No passing', 
            10:'No passing veh over 3.5 tons', 11:'Right-of-way at intersection', 12:'Priority road', 13:'Yield', 
            14:'Stop', 15:'No vehicles', 16:'Veh > 3.5 tons prohibited', 17:'No entry', 
            18:'General caution', 19:'Dangerous curve left', 20:'Dangerous curve right', 
            21:'Double curve', 22:'Bumpy road', 23:'Slippery road', 24:'Road narrows on the right', 
            25:'Road work', 26:'Traffic signals', 27:'Pedestrians', 28:'Children crossing', 
            29:'Bicycles crossing', 30:'Beware of ice/snow', 31:'Wild animals crossing', 
            32:'End speed + passing limits', 33:'Turn right ahead', 34:'Turn left ahead', 
            35:'Ahead only', 36:'Go straight or right', 37:'Go straight or left', 38:'Keep right', 
            39:'Keep left', 40:'Roundabout mandatory', 41:'End of no passing', 42:'End no passing veh > 3.5 tons'}

training_dataframe['ClassId']=training_dataframe['ClassId'].astype('str') # converting class type to string

training_dataframe = pd.concat([training_dataframe['Path'], training_dataframe['ClassId']], axis = 1) # simplify the dataframe by removing ROI and H/W information.
training_dataframe = utils.shuffle(training_dataframe) # Shuffling the dataframe 
training_dataframe.head() # check to ensure dataframe has been shuffled.

We will now build our image generators. We will start by defining a few generator hyperparameters (batch size and image shape) before creating a TensorFlow Image Data Generator (image_gen). We will use image_gen to create a training generator and validation generator, which allows for real time augmentation of training data as used. More information about the ImageDataGenerator can be found in [TensorFlow's API Documentation](https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image/ImageDataGenerator)

In [ ]:
# Define hyperparameters for the generators
BATCH_SIZE = 128
IMG_SIZE = (32, 32) # Fixing the image size is a required for our Neural Network. This size may be adjusted as needed.

# Create the training and validation generators
image_gen =  tf.keras.preprocessing.image.ImageDataGenerator(
    rotation_range = 10,
    width_shift_range = 0.1,
    height_shift_range = 0.1,
    shear_range = 0.15,
    zoom_range = 0.15,
    rescale = 1/255.0,
    horizontal_flip=False,
    vertical_flip=False,
    validation_split= 0.2,
    fill_mode = 'nearest')

# Training Generator
train_gen = image_gen.flow_from_dataframe(
    training_dataframe,
    directory=BASE_PATH,
    x_col='Path',
    y_col='ClassId',
    target_size=IMG_SIZE,
    color_mode="rgb",
    batch_size=BATCH_SIZE,
    subset="training",
)


# Validation Generator
val_gen = image_gen.flow_from_dataframe(
    training_dataframe,
    directory=BASE_PATH,
    x_col="Path",
    y_col='ClassId',
    target_size=IMG_SIZE,
    color_mode="rgb",
    batch_size=BATCH_SIZE,
    subset="validation",
)

# Test Training Generator with a sample
gen = next(train_gen)
print(gen[0].shape) # Confirm BATCH_SIZE, and IMG_SIZE
plt.imshow(gen[0][0,:,:,:]) # show sample image

With the generators in place, we are ready to design our Convolutional Neural Network (CNN) classifier. Typically, I like to use the TensorFlow Functional API to build neural networks as it allows flexibility in design, but for this notebook, I will use the Sequential Model. Additional information about the Functional API and Sequential Model can be found here:

[Functional API](https://www.tensorflow.org/guide/keras/functional) & 
[Sequential Model](https://www.tensorflow.org/guide/keras/sequential_model)

In [ ]:
# Hyperparameters for the Neural Networks
KERNEL_SIZE = (3,3)
POOL_SIZE = (2,2)
FILTERS = 16
dropout = 0.25
EPOCHS = 15
img_shape = (32,32, 3) # Defined above in the generator section. 



classifier = tf.keras.models.Sequential([    
    tf.keras.layers.Conv2D(filters=FILTERS, kernel_size=KERNEL_SIZE, activation='relu', input_shape=img_shape),
    tf.keras.layers.Conv2D(filters=FILTERS*2, kernel_size=KERNEL_SIZE, activation='relu'),
    tf.keras.layers.MaxPool2D(pool_size=POOL_SIZE),
    tf.keras.layers.BatchNormalization(axis=-1),
    
    tf.keras.layers.Conv2D(filters=FILTERS*4, kernel_size=KERNEL_SIZE, activation='relu'),
    tf.keras.layers.Conv2D(filters=FILTERS*8, kernel_size=KERNEL_SIZE, activation='relu'),
    tf.keras.layers.MaxPool2D(pool_size=POOL_SIZE),
    tf.keras.layers.BatchNormalization(axis=-1),
    
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(dropout),
    
    tf.keras.layers.Dense(NUM_CLASSES, activation='softmax')
])


# Plot the classifier model to a .png file 
tf.keras.utils.plot_model (classifier, to_file = 'classifier_structure.png', 
                           show_shapes = True)

# Compile the model using adam optimizer.
classifier.compile(loss = 'categorical_crossentropy', optimizer = 'adam',
            metrics = ['accuracy'])

# Print a summary of the model for quick observation
print(classifier.summary())

Now that the model has been built, we can go ahead and train it using our generators. As an FYI, the model has been plotted and saved to the file 'classifier_structure.png'.

In [ ]:
history = classifier.fit(x = train_gen, 
               epochs = EPOCHS, 
               validation_data = val_gen, 
               verbose = True)

classifier.save("GTSRB_CNN_Model.h5")

In [ ]:
pd.DataFrame(history.history).plot(figsize=(8, 5))
plt.grid(True)
plt.gca().set_ylim(0, 1)
plt.show()

These are pretty good results for a relatively simple CNN. Somewhere in the mid-to-upper 90% accuracy on the validation set is a great starting point. Now, we will want to test model on the official test data. As before, we will use our image_gen generator to build a sample to test with. 

In [ ]:
test_dataframe['ClassId'] = test_dataframe['ClassId'].astype('str')

# Test Generator
test_gen = image_gen.flow_from_dataframe(
    test_dataframe,
    directory=BASE_PATH,
    x_col='Path',
    y_col='ClassId',
    target_size=IMG_SIZE,
    color_mode="rgb",
    batch_size=BATCH_SIZE,
)

# Evaluate the classifier with data from our test generator
classifier.evaluate(test_gen)


Based on the generated test run, our classifier achieves about 95% accuraucy on the test data. This is pretty good performance, but certainly isn't state of the art compared to some of the published models (links available on the INI website, above). If I were going to further tweak/tune this model, I would start with modifying the hyperparameters for both the CNN and the generators. Since this is a single pass notebook (with only hand tuning of the results to get something reasonable), I will wrap it up for now. Thank you for taking the time to read through this notebook, I hope that you've found it interesting and/or helpful.